# Introduction

This notebook describes  the set of tools to building an LMDB of images and associated ground truths, run a SGD caffe solver, and test a set of images against a trained model.  Each tool consumes a JSON configuration file containing properties.

# Building an LMDB Image Set

The tool works with data under a provided data directory. The tool traverses a data subdirectories prefixed by 'DataSet_' looking for xls files.  The XLS files provide names of images, types of items in the image, polygons surrounding for each item in the image.    The XLS file must provide three columns:
 * Image File Name.  Format is 'name.postfix' where postfix is any type (e.g. tif, png, jpg)
 * Image Mode Type (classification)
 * Image Polygon.  Format mirrors this example: [142 52;143 51;...]
 The tool then pulls images from a image subdirectory corresponding to the image name from the XLS files.
 
An image may be referenced by multiple rows of XLS file, each with a different polygon.

A ground truth image is created for each image and associated rows from the XLS file.  The ground truth image is constructed int he same size as the original image with a black background.   Each polygon is filled into the ground truth image with a unique color assigned to each mode type (classification). 

The image and the ground truth are written to two LMDB databases.  The ground truth is written as a single channel image where each mode type is assigned a number from 1 to 255.  0 is background.  

The LMDB script creates six outputs:
1. png_raw - a directory of raw images, resized if requested
2. png_gt - a directory of ground truth images, using the same size of the raw image
3. raw_train - training LMDB image set
4. gt_train - training LMDB ground truth set
5. raw_test - testing LMDB image set
6. gt_test - testing LMDB ground truth set

## Configuring LMDB Creation

Sample configuration file is called create_db.json.

```json
{
   "dataDirectory": "/data/oirds",
   "imageDirectory": "png",
   "multiplicationFactor": 1,
   "labelIndex": 7,
   "resize": 128,
   "percentageToTest":0.18,
   "xlsInfoColumns" : [2,7,9],
   "modes": ["VEHICLE/CAR","VEHICLE/PICK-UP","VEHICLE/TRUCK","VEHICLE/UNKNOWN","VEHICLE/VAN"]
}
```

### Properties

| Property        | Description           |
| ------------- |:-------------:|
| dataDirectory   | the parent directory of the XLS files and images file |
| imageDirectory | subdirectory containing all image files |
| multiplicationFactor | number of times an image should be used to build test images, each with random changes to the original |
| labelIndex | force a single classification value in the ground truth image, otherwise the script assigns a number |
| percentageToTest| percentage of image data to use as a test vs. train.  Use a number between 0 and 1 |
| resize | force a resize of image and associated ground truth to a square image |
| xlsInfoColumns | the column numbers for the image file name, image mode and image polygon. |
| modes | the names of each mode type from the XLS file |

### Execution

```
  python create_train_dbs.py create_db.json
```


# Running the Solver


## Configuring Solver

Sample configuration file is called solver.json.

```json
{
   "modelFileName": "/data/oird_fcn/fcn-8s-pascalcontext.caffemodel",
   "prototxt" : "solver.prototxt",
   "netsurgery": true,
   "gpu": true,
   "steps":15000
}
```

### Properties

| Property        | Description           |
| ------------- |:-------------:|
| modelFileName   |the initial model to begin transfer learning|
| prototxt | solver prototxt |
| netsurgery | bi-linear interpolation on upsample layers |
| gpu | use the gpu |
| steps| number of iterations |


### Execution

```
  python solver_with_netsurgery.py solver.json
```

# Testing Labels

Testing and computation of statistics requires the ground truth data.  The configuration contains the same properties as the the creation of the LMDB, along with additional properties.

The network prototxt (deploy prototxt) defines the data layer as follows:
```
input: "data"
input_dim: 1
input_dim: 3
input_dim: 128
input_dim: 128
```
The final output layer should output height and width dimensions of the image in addition one channel per each class.  The testing script chooses the channel per pixel with the maximum value to choose the class for that pixel.

## Configuring the Tester

Sample configuration file is called solver.json.

```json
{
   "modelFileName": "/data/oird_fcn/fcn-8s-pascalcontext.caffemodel",
   "xlsInfoColumns" : [2,7,9],
   "modes": ["BACKGROUND","VEHICLE/CAR","VEHICLE/PICK-UP","VEHICLE/TRUCK","VEHICLE/UNKNOWN","VEHICLE/VAN"],
   "dataDirectory": "/data/oirds",
   "imageDirectory": "png",
   "meanarray": [112.0,112.0,112.0],
   "prototxt": "/data/oird_fcn/deploy.prototxt",
   "netsurgery": true,
   "gpu":true,
   "resize" :  128,
   "labelIndex": 7,
   "percentageToTest": 0.04,
   "networkOutputName": "score",
   "useCaffeImage":true,
   "useTransformer":true,
   "dumpBlobs":false
}
```

### Properties

| Property        | Description           |
| ------------- |:-------------:|
| dataDirectory   | the parent directory of the XLS files and images file |
| imageDirectory | subdirectory containing all image files |
| multiplicationFactor | number of times an image should be used to build test images, each with random changes to the original |
| labelIndex | force a single classification value in the ground truth image, otherwise the script assigns a number |
| percentageToTest| percentage of image data to use as a test the label.  Use a number between 0 and 1 |
| resize | force a resize of image and associated ground truth to a square image |
| xlsInfoColumns | the column numbers for the image file name, image mode and image polygon. |
| modes | the names of each mode type from the XLS file |
| meanarray| optional mean array, one for each channel.  Otherwise use  meanproto |
| meanproto | the full path name of the mean prototxt file containing the blob for means per every pixel (KXWXH) |
| prototxt | prototxt of the deployment network  This is the train network with a different output and no defined data layers|
| netsurgery | bi-linear interpolation on upsample layers |
| networkOutputName | the name of the output of the deployment network to use for scoring the best channel per pixel |
| useCaffeImage | Caffe images normalize the image prior to running caffe using scikit |
| useTransformer | if useCaffe is true, this should be true |
| dumpBlobs | creates image files for each convolution corresponding |
| imageName | name of a specific image referenced in the XLS files to test.  Otherwise a percentage will be tested |


### Execution

```
  python test_label.py test_label.json
```
